In [1]:
require 'nn'
require 'rnn'
require 'image'
require 'optim'
require 'parallel'
require 'cutorch'
require 'cunn'
require 'cunnx'
dl = require 'dataload'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function Tokenize(inputdic)
    --- This function tokenizes the words into a unigram dictionary
    local out = {}
    for k, v in pairs(inputdic) do
        if v ~= 0 then 
            if out[v] == nil then
                out[v] = 1
            else 
                out[v] = 1 + out[v]
            end
        end
    end
    return out
end

function rougeScores(genSummary, refSummary)
    local genTotal = 0
    local refTotal = 0
    local intersection = 0
    -- Inserting the missing keys
    for k, genCount in pairs(genSummary) do
        if refSummary[k] == nil then
            refSummary[k] = 0
        end
    end
    for k, refCount in pairs(refSummary) do
        local genCount = genSummary[k]
        if genCount == nil then 
            genCount = 0 
        end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
        genTotal = genTotal + genCount
    end

    recall = intersection / refTotal
    prec = intersection / genTotal
    if refTotal == 0 then
        recall = 0
    end 
    if genTotal == 0 then
        prec = 0
    end
    -- tmp = {intersection, refTotal, genTotal}
    if recall > 0 or prec > 0 then
        f1 = (2 * recall * prec) / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [3]:
function buildPredsummary(chosenactions, inputsentences, select_index)
    local summary = torch.zeros(inputsentences:size())
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(chosenactions, inputsentences, select_index)
    local n = inputsentences:size(1)
    local k = inputsentences:size(2)
    local summary = torch.zeros(inputsentences:size())
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
    --     This line didn't work for whatever reason...gives weird indexing...
    --     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

In [4]:
function buildTotalSummary(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            -- maxindex = 0
            -- for j = 1, totalPredsummary[i]:size(1) do 
            --     if totalPredsummary[i][j] == 0 then
            --         maxindex = maxindex + 1
            --     end
            -- end
            -- lenx = predsummary[i]:size(1)
            -- totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])

            minindex = 1
            for j = 1, totalPredsummary[i]:size(1) do 
                if totalPredsummary[i][j] > 0 then
                    minindex = minindex + 1
                end
            end
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])

        end
    end
end

-- function buildTotalSummaryFast(predsummary, inputTotalSummary)
--     totalPredsummary = inputTotalSummary:clone()
--     nps = predsummary:size(1)
--     n_l = inputTotalSummary:size(2)
--     indices = torch.linspace(1, n_l, n_l):long() 
--     for i=1, predsummary:size(1) do
--         if predsummary[i]:sum() > 0 then 
--             -- Finding the largest index with a zero
--             -- maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
--             -- totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
--             -- Finding the smallest index with a zero
--             minindex = torch.min(indices[torch.eq(inputTotalSummary[i], 0)])
--             lenx = predsummary[i]:size(1)
--             totalPredsummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])
--         end
--     end
--     return totalPredsummary
-- end

-- function buildTotalSummaryFast(predsummary, inputTotalSummary)
--     tmpSummary = inputTotalSummary:clone()
--     nps = predsummary:size(1)
--     n_l = inputTotalSummary:size(2)
--     indices = torch.linspace(1, n_l, n_l):long() 
--     for i=1, predsummary:size(1) do
--         if predsummary[i]:sum() > 0 then 
--             -- Finding the largest index with a zero
--             -- maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
--             -- totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
--             -- Finding the smallest index with a zero
--             minindex = torch.min(indices[torch.eq(tmpSummary[i], 0)])
--             lenx = predsummary[i]:size(1)
--             tmpSummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])
--         end
--     end
--     return tmpSummary
-- end

function buildTotalSummaryFast(predsummary, inputTotalSummary)
    nps = predsummary:size(1)
    n_l =  inputTotalSummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            -- Finding the largest index with a zero
            -- maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
            -- totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
            -- Finding the smallest index with a zero
            minindex = torch.min(indices[torch.eq(inputTotalSummary[i], 0)])
            lenx = predsummary[i]:size(1)
             inputTotalSummary[i][{{minindex, minindex + lenx - 1}}]:copy(predsummary[i])
        end
    end
end

In [5]:
-- Setting parameters
-- n = 10
n = 1
n_s = 10
k = 8
q = 5
a = 1
b = 1000
embDim = 50
gamma = 0.3
SKIP = 1
SELECT = 2
epsilon = 1
nepochs = 1000
fast = true

maskLayer = nn.MaskedSelect()

# Simulating the data

In [6]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end
-- Optimal predicted summary
trueSummary = torch.zeros(n, k * n_s)
-- Using this to generate the optimal actions
true_actions = {}

for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
    best_sentences = buildPredsummaryFast(true_actions[i], sentences[i], SELECT)
    buildTotalSummaryFast(best_sentences, trueSummary)
end

qTokens = {}
for i=1, n do
    qTokens[i] = Tokenize(trueSummary[i]:totable())
end

# Scoring the rougue metrics on the simulation

In [7]:
print(rougeScores(Tokenize(trueSummary[1]:totable()), Tokenize(trueSummary[1]:totable())))

1	1	1	


# Building the model

In [8]:
model = buildModel('lstm', b, embDim, 'f1', false, false)
-- model = buildModel('bow', b, embDim, 'f1', false, false)

params, gradParams = model:getParameters()
criterion = nn.MSECriterion()
maskLayer = nn.MaskedSelect()

Running LSTM model to learn f1	


# Scoring the model on the sentences

In [9]:
nepochs = 300
epsilon = 1
gamma = 0
base_explore_rate = 0.1
cuts = 4
delta = cuts/nepochs
end_baserate = torch.round(nepochs * 0.8)

In [10]:
optimParams = { learningRate = 1e-6}

totalPredsummary = {}
qValues = {}
qActions = {}
qPreds = {}
rewards = {}
lossfull = {}
rouguef1 = {}

totalPredsummary = torch.LongTensor(n, n_s * k):fill(0)

memsize = n * n_s
queryMemory = torch.zeros(memsize, q)
qActionMemory = torch.zeros(memsize, 2)
predSummaryMemory = torch.zeros(memsize, n_s * k)
sentenceMemory = torch.zeros(memsize, k)
qPredsMemory = torch.zeros(memsize, 2)
qValuesMemory = torch.zeros(memsize, 1)
rewardMemory = torch.zeros(memsize, 1)

--- Initializing thingss

for i = 1, n_s do
    qPreds[i] = torch.zeros(n, 2)
    qValues[i] = torch.zeros(n, 1) 
    qActions[i] = torch.zeros(n, 2)
    rewards[i] = torch.zeros(n, 1)
end 

for epoch=1, nepochs do
    --- Reset things at the start of each epoch
    for i=1, n_s do
        qPreds[i]:fill(0)
        qValues[i]:fill(0)
        qActions[i]:fill(0)
        rewards[i]:fill(0)
        totalPredsummary:fill(0)
    end

    for i=1, n_s do
        if torch.uniform(0, 1) <= epsilon then 
            qPreds[i]:copy(torch.rand(n, 2))
            -- Need to run a forward pass for the backward to work...wonky
            ignore = model:forward({sentences[i], queries, totalPredsummary})
        else 
            qPreds[i]:copy(model:forward({sentences[i], queries, totalPredsummary}) )
        end 
        if fast then 
            qMax, qindx = torch.max(qPreds[i], 2)  -- Pulling the best actions
            -- Here's the fast way to select the optimal action for each query
            qActions[i]:copy(qActions[i]:scatter(2, qindx, torch.ones(qPreds[i]:size())):clone())
            qValues[i]:copy(qMax)
            predsummary = buildPredsummaryFast(qActions[i], sentences[i], SELECT)
            buildTotalSummaryFast(predsummary, totalPredsummary)
        else 
            for j=1, n do
                if qPreds[i][j][SELECT] > qPreds[i][j][SKIP] then
                    qActions[i][j][SELECT] = 1
                    qValues[i][j]:fill(qPreds[i][j][SELECT])
                else
                    qActions[i][j][SKIP] = 1
                    qValues[i][j]:fill(qPreds[i][j][SKIP])
                end
            end
            predsummary = buildPredsummary(qActions[i], sentences[i], SELECT)
            buildTotalSummary(predsummary, totalPredsummary)
        end
        for j = 1, n do
            recall, prec, f1 = rougeScores( qTokens[j],
                                            Tokenize(totalPredsummary[j]:totable()))
            rewards[i][j]:fill(f1)
        end
        if i > 1 then
            -- Calculating change in rougue f1
            rewards[i]:copy(rewards[i] - rewards[i-1])
        end
        -- Update memory sequentially until it's full 
        qActionMemory[{{n * (i-1) + 1, n * i}}]:copy(qActions[i])
        predSummaryMemory[{{n * (i-1) + 1, n * i}}]:copy(totalPredsummary)
        sentenceMemory[{{n * (i-1) + 1, n * i}}]:copy(sentences[i])
        qPredsMemory[{{n * (i-1) + 1, n * i}}]:copy(qPreds[i])
        qValuesMemory[{{n * (i-1) + 1, n * i}}]:copy(qValues[i])
        queryMemory[{{n * (i-1) + 1, n * i}}]:copy(queries)
    end
    for i=1, n_s do
        if i  < n_s then
            rewardMemory[{{n * (i-1) + 1, n * i}}]:copy(rewards[i] + gamma * rewards[i + 1] )
        else
            rewardMemory[{{n * (i-1) + 1, n * i}}]:copy(rewards[i] )
        end
    end
    -- Adding back the delta for the last one
    rouguef1[epoch] = (rewards[n_s] + rewards[ n_s - 1] ):mean()

    loss = {}
    local dataloader = dl.TensorLoader({queryMemory, sentenceMemory, predSummaryMemory, 
                            qPredsMemory, qActionMemory, qValuesMemory}, rewardMemory)
    c = 1
    for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
        local function feval(params)
            gradParams:zero()
            if adapt then
                local ignore = model:forward({xin[1], xin[2], xin[3]})
                local predQOnActions = maskLayer:forward({qPredsMemory, actions_in}) 
                local ones = torch.ones(predQ:size(1)):resize(predQ:size(1))
                lossf = criterion:forward({qValuesMemory, predReg}, {reward, ones})
                local gradOutput = criterion:backward({qActionMemory, predReg}, {reward, ones})
                local gradMaskLayer = maskLayer:backward({qPredsMemory, qActionMemory}, gradOutput[1])
                model:backward({xin[1], xin[2], xin[3]}, {gradMaskLayer[1], gradOutput[2]})
            else 
                local ignore = model:forward({xin[1], xin[2], xin[3]})
                local predQOnActions = maskLayer:forward({xin[4], xin[5]:byte()}) 
                lossf = criterion:forward(predQOnActions, reward)
                local gradOutput = criterion:backward(predQOnActions, reward)
                local gradMaskLayer = maskLayer:backward({xin[4], xin[5]:byte()}, gradOutput)
                model:backward({xin[1], xin[2], xin[3]}, gradMaskLayer[1])
            end 
            return lossf, gradParams
        end
        --- optim.rmsprop returns \theta, f(\theta):= loss function
         _, lossv  = optim.rmsprop(feval, params, optimParams)
        loss[c] = lossv[1]
        c = c + 1
    end

    lossfull[epoch] = torch.Tensor(loss):sum() / #lossv
    if print_perf then
        print(
            string.format('epoch = %i; rougue = %.6f; epsilon = %.6f; loss = %.6f' , 
                epoch, rouguef1[epoch], epsilon, lossfull[epoch])
            )
    end

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
        if epoch > end_baserate then 
            base_explore_rate = 0.
        end
    else 
        epsilon = epsilon - delta
    end
    
end

In [11]:
Plot = require 'itorch.Plot'

loss = torch.Tensor(lossfull)
rougue = torch.Tensor(rouguef1)
indices = torch.linspace(1, loss:size(1), loss:size(1)):long() 
plot = Plot():line(indices, loss, 'red', 'hi'):title('Plot of loss'):draw()
plot = Plot():line(indices, rougue, 'blue', 'hi'):title('Plot of Rougue-F1'):draw()

In [12]:
rougeScores( Tokenize(trueSummary:resize(40):totable()), 
                Tokenize(totalPredsummary:totable()[1]))

0	0	0	


In [13]:
n = 10
-- nn.MaskedSelect():cuda()
maskLayer = nn.MaskedSelect()

x = torch.round(torch.rand(n, 2) * 5)

qMaxtrue, qindxtrue = torch.max(x, 2)
a = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(x:size())):byte()

In [20]:
SELECT, SKIP

2	1	


In [14]:
a, x, maskLayer:forward({x, a}) 

 1  0
 0  1
 0  1
 1  0
 1  0
 1  0
 1  0
 1  0
 1  0
 1  0
[torch.ByteTensor of size 10x2]

 2  0
 1  4
 2  5
 4  0
 2  2
 1  0
 3  3
 2  1
 3  1
 1  0
[torch.DoubleTensor of size 10x2]

 2
 4
 5
 4
 2
 1
 3
 2
 3
 1
[torch.DoubleTensor of size 10]



In [15]:
n = 10
maskLayer = nn.MaskedSelect():cuda()
ByteTensor = torch.CudaByteTensor

x = torch.round(torch.rand(n, 2) * 5):cuda()

In [16]:
qMaxtrue, qindxtrue = torch.max(x, 2)

In [17]:
torch.ones(x:size())

In [18]:
qindxtrue:cuda()

 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
[torch.DoubleTensor of size 10x2]



In [19]:
torch.zeros(n, 2):cuda():scatter(2, qindxtrue:cuda(), torch.ones(x:size()):cuda() )

[string "local f = function() return torch.zeros(n, 2)..."]:1: bad argument #3 to 'f' (Index tensor must have the same size as input tensor. at /tmp/luarocks_cutorch-scm-1-9089/cutorch/lib/THC/generic/THCTensorScatterGather.cu:116)
stack traceback:
	[C]: in function 'f'
	[string "local f = function() return torch.zeros(n, 2)..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main c